In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Intro

Kita akan melakukan analisis prediksi terhadap data promosi di TV, Radio, SocMed, dan Influencer - dan kaitannya terhadap Sales.
Bagaimana kita akan melakukannya?

1. Import required libraries
2. Read the dataframe
3. Is there any missing data? If yes, fill na data / impute na data
4. Is there any categorical variable? If yes, we need to perform encoding
5. Justify the independent and dependent variable
6. Split training and testing data
7. Model fit (training)
8. Model prediction (testing data)
9. Model evaluation (r-square, Root Mean Squared Error)
10. Recommendation!

**Catatan**: Pada contoh kali ini, kita tidak melakukan validasi asumsi matematis Regression (Linear) maupun Random Forest (apabila ada). Kita asumsikan asumsi-asumsi yang ada telah terpenuhi - dan kita lebih fokus ke *big picture* dan *interpretasinya*.

Untuk lebih detail mengenai linear regression assumption, [berikut](https://www.statology.org/linear-regression-assumptions/) merupakan salah satu ulasannya yang cukup sederhana.

# Import required libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# all the scikitlearn libraries here are used for the purpose of model analysis and evaluation

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

# Read the Dataframe

In [ ]:
df = pd.read_csv('/kaggle/input/dummy-advertising-and-sales-data/Dummy Data HSS.csv') # opening the csv data file

In [ ]:
df

# Is there any missing data?

In [ ]:
df.info()

In [ ]:
# if there's missing data, we have two options: REMOVE IT or IMPUTE IT.

In [ ]:
df = df.fillna(df.mean()) # update the existing 'df' with new code
df

In [ ]:
df.info()

# Is there any categorical data? 
Yes. Then, we have to perform encoding.

**We will use K-1 dummy variable strategy to avoid multicollinearity problem.** [See here](https://www.statisticssolutions.com/assumptions-of-linear-regression/) for more details.

In [ ]:
df = pd.get_dummies(df, drop_first = True)
df

In [ ]:
df.columns

In [ ]:
df = df[['TV', 'Radio', 'Social Media', 'Influencer_Mega',
       'Influencer_Micro', 'Influencer_Nano', 'Sales']]

df

In [ ]:
df.corr() 

# perhatikan disini bahwa terdapat korelasi yang cukup tinggi antara TV dan Radio, namun masih dibawah 0.9. Namun, kembali ke asumsi awal kita, kita lanjutkan saja terlebih dahulu.

In [ ]:
df.hist(figsize=(15,15))
plt.show() 

# perhatikan bahwa histogram dibawah ini merupakan non-parametrik (tidak normal). Namun, kembali ke asumsi awal kita, kita asumsikan saja normal terlebih dahulu.

In [ ]:
df.describe() # memahami descriptive statistics

# Justify the Independent and Dependent Variable (X & Y)

In [ ]:
x = df.iloc[:,0:-1].values
y = df.iloc[:,-1:].values

print(x)
print(y)

# Split Training and Testing Data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1000000)

# Train the Data

In [ ]:
lr_regressor = LinearRegression()
lr_regressor.fit(x_train, y_train)

Sampai point ini, kita sudah melakukan training ke model linear regression kita.

# Predict the Data

In [ ]:
lr_y_pred = lr_regressor.predict(x_test)

Sampai point ini, kita sudah punya data Sales REAL dan Sales hasil PREDIKSI.
Kita kemudian ingin tau, seberapa akurat model kita, dan seberapa besar errornya.

Untuk melihat kurang lebih preview Sales REAL dan Sales PREDIKSI, kita lakukan sebagai berikut:

In [ ]:
print(y_test)
print("---")
print(lr_y_pred)

# Model Evaluation

Jadi ada beberapa hal yang bisa kita evaluasi dalam model Linear Regression kita:

1. R-square
2. Root Mean Square Error
3. Regression Coefficient
4. Regression Intercept

In [ ]:
r2_score(y_test, lr_y_pred)

In [ ]:
mse(y_test, lr_y_pred)**0.5

Sehingga, berdasarkan r-square and RMSE:

1. 99% perubahan yang terjadi pada Sales, dipengaruhi oleh variabel independent kita
2. Terdapat error sekitar 6.7 point antara Sales PREDIKSI dan Sales REAL data yang kita punya

In [ ]:
df['Sales'].describe()

In [ ]:
print(lr_regressor.coef_)
print(lr_regressor.intercept_)

In [ ]:
df.head()

Apa kesimpulan yang bisa diambil dari koefisien yang ada?

Sales = 3.51 TV + 0.06 Radio + 0.06 Socmed + 0.24 Mega + 0.41 Micro + 0.139 Nano - 0.11

# Recommendation


1. Prioritaskan TV dan Radio
2. Evaluasi Radio dan Socmed kita
3. Fokus ke influencer Micro

# Kita coba bandingkan Linear Regression dengan Random Forest Regression

Kita telah mengetahui akurasi Linear Regression dengan asumsi yang kita anggap terpenuhi (kenyataannya tidak) pada contoh ini. Sebagai pembanding, maka kita akan menggunakan Random Forest Regression yang tidak memerlukan asumsi seperti layaknya Linear Regression.

In [ ]:
rf_regressor = RandomForestRegressor()
rf_regressor.fit(x_train, y_train)

y_rf_pred = rf_regressor.predict(x_test)

print(r2_score(y_test, y_rf_pred))
print(mse(y_test, y_rf_pred)**0.5)

Jadi kita bisa melihat bahwa:

* Linear Regression menghasilkan: 0.988 dan 9.98 (r-squared dan RMSE)
* Random Forest Regression menghasilkan: 0.992 dan 7.84 (r-squared dan RMSE)

Kita bisa menyimpulkan bahwa untuk melakukan prediksi dengan data yang ada, Random Forest memberikan akurasi lebih tinggi daripada Linear Regression, dan error yang lebih kecil dibandingkan Linear Regression.

Sehingga, kalau kita mau melakukan prediksi atas Sales, maka kita bisa consider menggunakan Random Forest Regression dibandingkan Linear Regression.

# Catatan Konteks untuk Rekomendasi

1. Kita telah melakukan analisis data untuk mengevaluasi dan menentukan arah promosi kedepan. 

2. Kita tidak boleh hanya bergantung terhadap angka-angka yang didapatkan, tetapi juga kepada konteks makro/mikro yang riil diluar apa yang data ini telah berikan. 
    Misalnya, mengapa Socmed kita tidak berperan baik? Apakah misalnya kita salah menentukan channel socmed? Apakah Socmed yang kita tentukan sudah sesuai dengan target market kita? 
    Mengapa radio tidak berperan baik? Apakah market kita memang mendengarkan radio dalam kesehariannya? 
    Jadi terdapat aksi evaluatif maupun prediktif terhadap data dan lingkungan yang ada pada bisnis yang kita sedang pelajari.

3. Hal inilah yang perlu dilacak kembali, sehingga kita bisa memiliki perspektif kualitatif dan kuantitatif dalam menentukan arah kita kedepan.